**Кулешов Иван AML-14**

Предварительно про PyTorch:
* [Про тензоры в pytorch](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/tensor_tutorial.ipynb)
* [Про автоматическое дифференцирование и что такое .backwards()](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/autograd_tutorial.ipynb)
* [Очень простая нейронка на pytorch](https://colab.research.google.com/drive/1RsZvw4KBGn5U5Aj5Ak7OG2pHx6z1OSlF)

# Домашнее задание № 2 по теме "Классификация в АОТ"


## Сделать классификацию данных fakenews
#### Используя ноутбук занятия (также размещен в папке Materials) и данные fakenews, 3 раза разными способами получить на задаче классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models.word2vec import Word2Vec

In [ ]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2022-09-30 12:19:30--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.06s   

2022-09-30 12:19:31 (20.3 MB/s) - ‘Constraint_Train.csv’ saved [1253562/1253562]



In [ ]:
df = pd.read_csv('Constraint_Train.csv')

In [ ]:
df.head()

In [ ]:
df.label.value_counts()

real    3360
fake    3060
Name: label, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6420 entries, 0 to 6419
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6420 non-null   int64 
 1   tweet   6420 non-null   object
 2   label   6420 non-null   object
dtypes: int64(1), object(2)
memory usage: 150.6+ KB


**Вывод:** классы почти сбалансированны, нулевых значений нет

#### Токенизация текста

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Подготовим два корпуса данных: просто токенизированный текст и лемматизированный текст с удалением стоп-слов. Каждый из корпусов будет затем использоваться в трех способах классификации.

In [ ]:
# простая токенизация
tweets = [word_tokenize(tweet.lower()) for tweet in df.tweet]

In [ ]:
tweets[0]

['the',
 'cdc',
 'currently',
 'reports',
 '99031',
 'deaths',
 '.',
 'in',
 'general',
 'the',
 'discrepancies',
 'in',
 'death',
 'counts',
 'between',
 'different',
 'sources',
 'are',
 'small',
 'and',
 'explicable',
 '.',
 'the',
 'death',
 'toll',
 'stands',
 'at',
 'roughly',
 '100000',
 'people',
 'today',
 '.']

Лемматизатор для английского языка

In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
tweets_lemma = [
    [lemmatizer.lemmatize(word) for word in words 
     if not word in stopwords.words("english") 
     ] 
    for words in tqdm(tweets) # Перебираем каждый твит - list
    ]

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
tweets_lemma[0]

['cdc',
 'currently',
 'report',
 '99031',
 'death',
 '.',
 'general',
 'discrepancy',
 'death',
 'count',
 'different',
 'source',
 'small',
 'explicable',
 '.',
 'death',
 'toll',
 'stand',
 'roughly',
 '100000',
 'people',
 'today',
 '.']

## Способ 1 - простая векторизация CountVectorizer

### Сначала попробуем на биграммах

In [ ]:
count_vec1 = CountVectorizer(ngram_range=(2, 2))
count_vec2 = CountVectorizer(ngram_range=(2, 2))

X_tweet = count_vec1.fit_transform([' '.join(tweet) for tweet in tweets])
X_lemma = count_vec2.fit_transform([' '.join(tweet) for tweet in tweets_lemma])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tweet, df.label, test_size=0.2, random_state=66, stratify=df.label)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_lemma, df.label, test_size=0.2, random_state=66, stratify=df.label)

Воспользуемся логистической регрессией для классификации:

In [ ]:
model = LogisticRegression(random_state = 66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Простой векторизованный текст:")
print(classification_report(y_test, predicted))

model = LogisticRegression(random_state = 66)
model.fit(X_train1, y_train1)
predicted = model.predict(X_test1)
print("Лемматизированный текст:")
print(classification_report(y_test1, predicted))

Простой векторизованный текст:
              precision    recall  f1-score   support

        fake       0.84      0.92      0.88       612
        real       0.92      0.85      0.88       672

    accuracy                           0.88      1284
   macro avg       0.88      0.88      0.88      1284
weighted avg       0.88      0.88      0.88      1284

Лемматизированный текст:
              precision    recall  f1-score   support

        fake       0.82      0.90      0.86       612
        real       0.90      0.82      0.86       672

    accuracy                           0.86      1284
   macro avg       0.86      0.86      0.86      1284
weighted avg       0.86      0.86      0.86      1284



### Классификация на униграммах

In [ ]:
count_vec1 = CountVectorizer(ngram_range=(1, 1))
count_vec2 = CountVectorizer(ngram_range=(1, 1))

X_tweet = count_vec1.fit_transform([' '.join(tweet) for tweet in tweets])
X_lemma = count_vec2.fit_transform([' '.join(tweet) for tweet in tweets_lemma])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tweet, df.label, test_size=0.2, random_state=66, stratify=df.label)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_lemma, df.label, test_size=0.2, random_state=66, stratify=df.label)

Воспользуемся логистической регрессией для классификации:

In [ ]:
model = LogisticRegression(random_state = 66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print("Простой векторизованный текст:")
print(classification_report(y_test, predicted))

model = LogisticRegression(random_state = 66)
model.fit(X_train1, y_train1)
predicted = model.predict(X_test1)
print("Лемматизированный текст:")
print(classification_report(y_test1, predicted))

Простой векторизованный текст:
              precision    recall  f1-score   support

        fake       0.92      0.94      0.93       612
        real       0.94      0.93      0.94       672

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284

Лемматизированный текст:
              precision    recall  f1-score   support

        fake       0.91      0.93      0.92       612
        real       0.94      0.92      0.93       672

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



Для некоторой альтернативы попробуем другую модель классификации:

In [ ]:
model = SVC(random_state=66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.92      0.92       612
        real       0.93      0.92      0.93       672

    accuracy                           0.92      1284
   macro avg       0.92      0.92      0.92      1284
weighted avg       0.92      0.92      0.92      1284



**Выводы:** итак, можно сделать следующие наблюдения:
- для классификации лучше брать полный текст, без лемматизации и удаления стоп-слов;
- классификация на униграммах предпочтительнее, чем на биграммах;
- в качестве модели логистическая регрессия эффективнее метода опорных векторов.

## Способ 2: Векторизация с помощью tfidf

Как показали эксперименты выше, лучше взять униграммы и не пользоваться лемматизированным вариантом:

In [ ]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 1))

X_tweet = tfidf_vec.fit_transform([' '.join(tweet) for tweet in tweets])

X_train, X_test, y_train, y_test = train_test_split(X_tweet, df.label, test_size=0.2, random_state=66, stratify=df.label)

Классификация на основе текста, векторизованного методом tfidf:

In [ ]:
model = LogisticRegression(random_state = 66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.90      0.93      0.92       612
        real       0.93      0.91      0.92       672

    accuracy                           0.92      1284
   macro avg       0.92      0.92      0.92      1284
weighted avg       0.92      0.92      0.92      1284



**Вывод:** метод векторизации слов tf-idf показал результат хуже чем CountVectorizer.

## Способ 3: Векторизацию с помощью word2vec

Построим модель:

In [ ]:
#  size - размер вектора, window -размер окна наблюдения,
#  min_count - мин. частотность слова в корпусе, которое мы берем,
#  sg - используемый алгоритм обучение (0 - CBOW, 1 - Skip-gram))

model_tweets = Word2Vec(tweets, workers=4, size=300, min_count=3, window=5, iter=50)

Самые близкие слова к слову Russia:

In [ ]:
model_tweets.wv.most_similar('russia')

[('putin', 0.6421512961387634),
 ('vladimir', 0.6004921793937683),
 ('donated', 0.5860686302185059),
 ('honjo', 0.5710262060165405),
 ('dr', 0.5694546103477478),
 ('usa', 0.5644663572311401),
 ('injection', 0.5575618743896484),
 ('russian', 0.5463930368423462),
 ('whistleblower', 0.5462302565574646),
 ('antonio', 0.5445864796638489)]

In [ ]:
model_tweets.init_sims(replace=True) # для экономии памяти

Функция, возвращающая эмбеддинг:

In [ ]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(300)
    return result

In [ ]:
features = [get_text_embedding(' '.join(text)) for text in tqdm(tweets)]

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.2, random_state=66, stratify=df.label)

In [ ]:
model = LogisticRegression(random_state = 66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.93      0.93       612
        real       0.94      0.93      0.93       672

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



Видоизменим алгоритм, поставим на основе Skip-gram:

In [ ]:
model_tweets = Word2Vec(tweets, workers=4, size=300, min_count=3, window=5, iter=50, sg=1)
model_tweets.init_sims(replace=True) # для экономии памяти

features = [get_text_embedding(' '.join(text)) for text in tqdm(tweets)]

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.2, random_state=66, stratify=df.label)

model = LogisticRegression(random_state = 66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.93      0.93      0.93       612
        real       0.93      0.94      0.94       672

    accuracy                           0.93      1284
   macro avg       0.93      0.93      0.93      1284
weighted avg       0.93      0.93      0.93      1284



Наконец, может, что-то изменится для лемматизированного текста? —

In [ ]:
model_tweets = Word2Vec(tweets_lemma, workers=4, size=300, min_count=3, window=5, iter=50, sg=1)
model_tweets.init_sims(replace=True) # для экономии памяти

In [ ]:
features = [get_text_embedding(' '.join(text)) for text in tqdm(tweets_lemma)]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.2, random_state=66, stratify=df.label)

model = LogisticRegression(random_state = 66)
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.95      0.94      0.94       612
        real       0.94      0.95      0.95       672

    accuracy                           0.95      1284
   macro avg       0.95      0.95      0.95      1284
weighted avg       0.95      0.95      0.95      1284



**Вывод:** Классификация на основе эмбеддингов слов word2vec, обученных на простом токенизированном тексте, оказалась менее точной, чем метод CountVectorizer.
Однако, модель word2vec на основе алгоритма skip-gram оказался чуточку лучше; а самым точным из всех испробованных подходов показала модель на эмбеддингах слов, обученных на лемматизированном тексте, хотя для всех моделей ранее лемматизированный текст уступал обычному без обработки.

## Нейронные сети для предсказания тональности: PyTorch + LSTM

In [ ]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для максимальной длины предложений.

In [ ]:
max_len = len(max(tweets, key=len))
max_len

1012

Это слишком много. Но какая длина обычно?

In [ ]:
from collections import Counter
fd = Counter([len(tokens) for tokens in tweets_lemma])
fd.most_common(10)

[(17, 258),
 (18, 242),
 (19, 239),
 (16, 230),
 (12, 230),
 (21, 227),
 (14, 226),
 (13, 223),
 (20, 214),
 (15, 213)]

Зададим максимум 200. Возьмём те же w2v эмбеддинги.

In [ ]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(300))
        else:
            result.append(np.zeros(300))
    return result

In [ ]:
features = [get_word_embedding(text, 200) for text in tqdm(tweets)]

  0%|          | 0/6420 [00:00<?, ?it/s]

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=66, stratify=labels)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

Импортируем библиотеки PyTorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(300, 100, dropout=0.5)
        self.out = nn.Linear(100, 1)

    def forward(self, x):

      embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
      prediction = torch.nn.functional.sigmoid(self.out(longterm))
      return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(300, 100, dropout=0.5)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


Переводим данные в torch тензоры:

In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape, targets.shape

(torch.Size([4301, 200, 300]), torch.Size([4301]))

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
criterion = nn.BCELoss()

In [ ]:
#функция расчета точности предсказания accuracy
def accuracy(probs, target):
    correct = (probs > 0.5) == target
    accuracy = correct.sum().item() / len(target)
    return accuracy

Функция для тренировки нейросети:

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for b in tqdm(range(0, in_data.shape[0])):
        for i in range(0, 200, batch_size): #tqdm(range(0, in_data.shape[0], batch_size)):
            batch_x = in_data[b:b+1, i:i + batch_size]
            batch_y = targets[b:b+1]
            optimizer.zero_grad()
            output = net(batch_x)
            loss = criterion(output.reshape(-1), batch_y)
            loss.backward()
            optimizer.step()
    print(loss)

Процесс тренировки:

In [ ]:
train_one_epoch(in_data, targets)

  0%|          | 0/4301 [00:00<?, ?it/s]

tensor(0.6107, grad_fn=<BinaryCrossEntropyBackward0>)


In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net.forward(in_data_test).reshape(-1)

In [ ]:
result = (output > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

0.7413874469089193

## Обучение нейросети

In [ ]:
import os
from functools import partial
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
df.label = (df.label == 'real').astype(int).to_list()

Формируем генератор наших данных, отдающий информацию посимвольно:

In [ ]:
class MyDataset(Dataset):
    def __init__(self, ds):
        super(MyDataset,self).__init__()
        self.labels = ds["label"].values
        self.data = ds["tweet"].values

    def __getitem__(self, idx):
        labels = self.labels[idx]
        data = self.data[idx]
        return labels, data

    def __len__(self):
        return len(self.data)

Первые 50 символов

In [ ]:
ds = MyDataset(df)
label, text = next(iter(ds))

Токенизируем текст

In [ ]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter,tokenizer):
    for _,text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(ds,tokenizer))

In [ ]:
def text_pipeline(text,vocab,tokenizer):
    tokens = tokenizer(text)
    vocab_list = []
    for t in tokens:
        vocab_list.append(vocab[t])
    return vocab_list

text_pipeline = partial(text_pipeline,vocab=vocab,tokenizer=tokenizer)

In [ ]:
def collate_batch(batch,text_pipeline,max_seq_len,device):
    data = torch.zeros((len(batch),max_seq_len),dtype=torch.int64)
    labels = torch.zeros((len(batch)),dtype=torch.int64)
    for index,data_tuple in enumerate(batch):
        processed_text = torch.tensor(text_pipeline(data_tuple[1]), dtype=torch.int64)
        data[index,:len(processed_text)] = processed_text[:max_seq_len]
        labels[index] = data_tuple[0]
    return labels.to(device), data.to(device)

max_seq_len = 1000
col_fn = partial(collate_batch,text_pipeline=text_pipeline,max_seq_len=max_seq_len,device=device)

Формируем архитектуру нейросети, на основе эмбедингов слов, модели LSTM:

In [ ]:
class Net(torch.nn.Module):
    def __init__(self,vocab_length,embedding_dim,seq_len,num_layers,num_hidden):
        super(Net,self).__init__()
        self.embedding = nn.Embedding(vocab_length,embedding_dim,max_norm=True)
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=num_hidden,num_layers=num_layers,batch_first=True)
        self.fc = nn.Linear(seq_len*num_hidden,2)
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = torch.reshape(x, (x.size(0),-1,))
        x = self.fc(x)
        return torch.nn.functional.log_softmax(x,dim=-1)

Инициализация модели, возьмем размер эмбеддинга = 50 и количество скрытых слоев = 50.

In [ ]:
batch_size = 256
model = Net(len(vocab),embedding_dim=50,seq_len=max_seq_len,num_layers=1,num_hidden=50)
model.to(device)
dl = DataLoader(ds,batch_size=batch_size,shuffle=True,num_workers=0,collate_fn=col_fn)

Определяем алгоритм оптимизации, функцию потерь:

In [ ]:
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

num_train = int(len(dl)*0.7)
num_valid = len(dl) - num_train
training_data, validation_data = random_split(dl, [num_train,num_valid])
datasets = {"Training":training_data.dataset, "Validation":validation_data.dataset}

Обучаем модель:

In [ ]:
num_epoch = 10
val_losses = [np.inf]
no_improvement = 0
for epoch in range(num_epoch):
    for d in datasets:
        if d == "Training":
            model.train(True)
        else:
            model.train(False)
        dataset = datasets[d]
        total_pts = 0
        running_loss, running_acc = 0.0, 0.0
        for i, sample in enumerate(dataset):
            labels,data = sample
            optimizer.zero_grad()
            out = model(data)
            _, pred = torch.max(out, 1)
            num_correct = (pred == labels).sum()
            loss = criterion(out,labels)
            if d == "Training":
                loss.backward()
                optimizer.step()
            running_loss += loss.item()
            running_acc  += num_correct.data.item()
            total_pts += len(sample[0])

        print("Epoch {}, {} Loss: {}, Accuracy: {}".format(epoch + 1, d, running_loss / i, running_acc / total_pts * 100))
        if d == "Validation":
            val_loss = running_loss / i
            if all(val_losses < np.array(val_loss)):
                no_improvement = 0
            else:
                no_improvement += 1
            val_losses.append(val_loss)
            if no_improvement == 3:
                break

Epoch 1, Training Loss: 1.1142053961753846, Accuracy: 51.822429906542055
Epoch 1, Validation Loss: 0.6978230500221252, Accuracy: 47.71028037383178
Epoch 2, Training Loss: 0.6169141864776612, Accuracy: 70.06230529595015
Epoch 2, Validation Loss: 0.5400383865833283, Accuracy: 74.50155763239876
Epoch 3, Training Loss: 0.4797250235080719, Accuracy: 76.88473520249221
Epoch 3, Validation Loss: 0.3766628009080887, Accuracy: 84.12772585669782
Epoch 4, Training Loss: 0.2706152337789536, Accuracy: 88.84735202492212
Epoch 4, Validation Loss: 0.17562100678682327, Accuracy: 93.3644859813084
Epoch 5, Training Loss: 0.1410677008330822, Accuracy: 94.47040498442367
Epoch 5, Validation Loss: 0.0922083842754364, Accuracy: 96.91588785046729
Epoch 6, Training Loss: 0.07691474251449108, Accuracy: 97.35202492211839
Epoch 6, Validation Loss: 0.043566261418163776, Accuracy: 98.42679127725856
Epoch 7, Training Loss: 0.037155736982822415, Accuracy: 98.62928348909658
Epoch 7, Validation Loss: 0.02486510505899787,

**Вывод:** нейросеть, обученная на 10 эпохах, дала неплохой результат, опередив по метрике точности модель классификации логистической регрессии на тексте, векторизованного word2vec.